In [3]:
#Import all necessary libraries
from pyspark.sql import SparkSession

In [4]:
# Creating a SparkSession in Python
spark = SparkSession.builder.appName('sales')\
    .config('spark.driver.extraClassPath', '/usr/lib/jvm/java-17-openjdk-amd64/lib/postgresql-42.5.0.jar')\
    .getOrCreate()


In [5]:
# Read json file
sales_data = spark.read.json('data/sales_records.json')    

sales_data.show()

+--------------------+---------------+----------+---------+--------------+--------------------+-------------+----------+----------+------------+-------------+---------+----------+----------+---------------+
|             Country|      Item Type|Order Date| Order ID|Order Priority|              Region|Sales Channel| Ship Date|Total Cost|Total Profit|Total Revenue|Unit Cost|Unit Price|Units Sold|_corrupt_record|
+--------------------+---------------+----------+---------+--------------+--------------------+-------------+----------+----------+------------+-------------+---------+----------+----------+---------------+
|          Azerbaijan|         Snacks| 10/8/2014|535113847|             C|Middle East and N...|       Online|10/23/2014|  91008.96|    51500.76|    142509.72|    97.44|    152.58|       934|           null|
|              Panama|      Cosmetics| 2/22/2015|874708545|             L|Central America a...|      Offline| 2/27/2015|1198414.83|   791282.37|   1989697.20|   263.33|    

In [8]:
sales_data.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Total Cost: string (nullable = true)
 |-- Total Profit: string (nullable = true)
 |-- Total Revenue: string (nullable = true)
 |-- Unit Cost: string (nullable = true)
 |-- Unit Price: string (nullable = true)
 |-- Units Sold: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)



1. Find the total cost, total revenue, total profit on the basis of each region.

In [29]:
from pyspark.sql import SparkSession , functions as fun, Window as Wd
sales_data.groupBy('Region').agg(fun.sum('Total Cost'), fun.sum('Total Revenue'), fun.sum('Total Profit')).show()

+--------------------+--------------------+--------------------+--------------------+
|              Region|     sum(Total Cost)|  sum(Total Revenue)|   sum(Total Profit)|
+--------------------+--------------------+--------------------+--------------------+
|Middle East and N...|1.194003115569000...|1.691834583652000...| 4.978314680829999E9|
|Australia and Oce...|7.5260986623499975E9|1.070152222373000...|3.1754235613800006E9|
|              Europe|2.415937816666000...|3.423977049931005E10|1.008039233265000...|
|  Sub-Saharan Africa|2.465031758511998E10|3.495487197082999E10|1.030455438570999...|
|Central America a...|1.026651964281999...|1.455373016529000...|     4.28721052247E9|
|       North America|2.0644507166499999E9|2.9370023334900002E9| 8.725516168399999E8|
|                Asia|   1.358588970306E10|   1.929340121982E10| 5.707511516759997E9|
|                null|                null|                null|                null|
+--------------------+--------------------+-----------

2. Find the Item List on the basis of each country.

In [30]:
sales_data.withColumn('Itemlist', fun.collect_set('Item Type').over(Wd.partitionBy('Country'))).select('Country', 'Itemlist').distinct().show()


+--------------------+--------------------+
|             Country|            Itemlist|
+--------------------+--------------------+
|                null|                  []|
|         Afghanistan|[Beverages, Perso...|
|             Albania|[Beverages, Perso...|
|             Algeria|[Beverages, Perso...|
|             Andorra|[Beverages, Perso...|
|              Angola|[Beverages, Perso...|
|Antigua and Barbuda |[Beverages, Perso...|
|             Armenia|[Beverages, Perso...|
|           Australia|[Beverages, Perso...|
|             Austria|[Beverages, Perso...|
|          Azerbaijan|[Beverages, Perso...|
|             Bahrain|[Beverages, Perso...|
|          Bangladesh|[Beverages, Perso...|
|            Barbados|[Beverages, Perso...|
|             Belarus|[Beverages, Perso...|
|             Belgium|[Beverages, Perso...|
|              Belize|[Beverages, Perso...|
|               Benin|[Beverages, Perso...|
|              Bhutan|[Beverages, Perso...|
|Bosnia and Herzeg...|[Beverages

3. Find the total number of items sold in each country.

In [31]:
sales_data.groupBy('Country').agg(fun.sum('Units Sold').alias('Total number of items sold')).show()

+-----------+--------------------------+
|    Country|Total number of items sold|
+-----------+--------------------------+
|       Chad|                 2660461.0|
|     Russia|                 2579558.0|
|      Yemen|                 2966519.0|
|    Senegal|                 2716010.0|
|     Sweden|                 2698756.0|
|   Kiribati|                 2555774.0|
|    Eritrea|                 2552497.0|
|Philippines|                 2610149.0|
|   Djibouti|                 2699545.0|
|      Tonga|                 2565238.0|
|  Singapore|                 2693579.0|
|   Malaysia|                 2587267.0|
|       Fiji|                 2613373.0|
|     Turkey|                 2732629.0|
|     Malawi|                 2645975.0|
|       Iraq|                 2765491.0|
|    Germany|                 2502470.0|
|    Comoros|                 2556790.0|
|   Cambodia|                 2946963.0|
|Afghanistan|                 2805640.0|
+-----------+--------------------------+
only showing top

4.  Find the top five famous items list on the basis of each region.(Consider units sold while doing this.)

In [32]:
famous_item_in_region = sales_data.groupBy('Region', 'Item Type').agg(fun.sum('Units Sold').alias('Total Units Sold')).orderBy('Region', 'Total Units Sold')
famous_item_in_region.withColumn('rank', fun.rank().over(Wd.partitionBy('Region').orderBy(fun.col('Total Units Sold').desc()))).where('rank <= 5').show()


+--------------------+---------------+----------------+----+
|              Region|      Item Type|Total Units Sold|rank|
+--------------------+---------------+----------------+----+
|                null|           null|            null|   1|
|                Asia|         Cereal|       6397658.0|   1|
|                Asia|         Snacks|       6330265.0|   2|
|                Asia|Office Supplies|       6187421.0|   3|
|                Asia|     Vegetables|       6121703.0|   4|
|                Asia|        Clothes|       6117405.0|   5|
|Australia and Oce...|  Personal Care|       3616014.0|   1|
|Australia and Oce...|     Vegetables|       3569149.0|   2|
|Australia and Oce...|         Cereal|       3535286.0|   3|
|Australia and Oce...|      Beverages|       3436051.0|   4|
|Australia and Oce...|        Clothes|       3393563.0|   5|
|Central America a...|      Cosmetics|       4814795.0|   1|
|Central America a...|        Clothes|       4675554.0|   2|
|Central America a...|  

5. Find all the regions and their famous sales channels.

In [34]:
sales_data.withColumn('Sales Channels', fun.collect_set('Sales Channel').over(Wd.partitionBy('Region'))).select('Region', 'Sales Channels').distinct().show()


+--------------------+-----------------+
|              Region|   Sales Channels|
+--------------------+-----------------+
|                null|               []|
|                Asia|[Online, Offline]|
|Australia and Oce...|[Online, Offline]|
|Central America a...|[Online, Offline]|
|              Europe|[Online, Offline]|
|Middle East and N...|[Online, Offline]|
|       North America|[Online, Offline]|
|  Sub-Saharan Africa|[Online, Offline]|
+--------------------+-----------------+



6. Find the list of countries and items and their respective units.

In [35]:
sales_data.groupBy('Country', 'Item Type').agg(fun.sum('Units Sold').alias('Units Sold')).orderBy('Country').show()

+-----------+---------------+----------+
|    Country|      Item Type|Units Sold|
+-----------+---------------+----------+
|       null|           null|      null|
|Afghanistan|      Cosmetics|  217192.0|
|Afghanistan|         Cereal|  256936.0|
|Afghanistan|     Vegetables|  219937.0|
|Afghanistan|         Snacks|  237350.0|
|Afghanistan|Office Supplies|  166911.0|
|Afghanistan|           Meat|  273402.0|
|Afghanistan|        Clothes|  220429.0|
|Afghanistan|      Baby Food|  232084.0|
|Afghanistan|  Personal Care|  255956.0|
|Afghanistan|      Beverages|  206154.0|
|Afghanistan|         Fruits|  257336.0|
|Afghanistan|      Household|  261953.0|
|    Albania|         Cereal|  215238.0|
|    Albania|      Baby Food|  191480.0|
|    Albania|        Clothes|  250884.0|
|    Albania|           Meat|  266123.0|
|    Albania|      Beverages|  188950.0|
|    Albania|         Snacks|  210384.0|
|    Albania|Office Supplies|  236822.0|
+-----------+---------------+----------+
only showing top

7. In 2013, identify the regions which sold maximum and minimum units of item type Meat.

In [37]:
sales_2013=sales_data.withColumn('year', fun.substring('Order Date', -4, 4)).where(fun.col('year') == '2013')

sales_2013.where(sales_data['Item Type'] == 'Meat').groupBy('Region', 'Item Type').agg(F.sum('Units Sold').alias('Units Sold')).orderBy('Units Sold').show()


+--------------------+---------+----------+
|              Region|Item Type|Units Sold|
+--------------------+---------+----------+
|       North America|     Meat|  106193.0|
|Australia and Oce...|     Meat|  449346.0|
|Central America a...|     Meat|  615706.0|
|Middle East and N...|     Meat|  745940.0|
|                Asia|     Meat|  956367.0|
|              Europe|     Meat| 1468932.0|
|  Sub-Saharan Africa|     Meat| 1491277.0|
+--------------------+---------+----------+



8. List all the items whose unit cost is less than 500.

In [33]:
sales_data.filter(sales_data["Unit Cost"] < 500).select('Item Type', 'Unit Cost').distinct().show()

+-------------+---------+
|    Item Type|Unit Cost|
+-------------+---------+
|   Vegetables|    90.93|
|    Cosmetics|   263.33|
|       Cereal|   117.11|
|    Baby Food|   159.42|
|       Snacks|    97.44|
|Personal Care|    56.67|
|    Beverages|    31.79|
|       Fruits|     6.92|
|      Clothes|    35.84|
|         Meat|   364.69|
+-------------+---------+



9. Find the total cost, revenue and profit of each year.

In [38]:
sales_1 = sales_data.withColumn('Year', fun.substring('Order Date', -4, 4))\

sales_1.groupBy('Year').agg(fun.sum('Total Cost'), fun.sum('Total Revenue'), fun.sum('Total Profit')).orderBy('Year').show()

+----+--------------------+--------------------+--------------------+
|Year|     sum(Total Cost)|  sum(Total Revenue)|   sum(Total Profit)|
+----+--------------------+--------------------+--------------------+
|null|                null|                null|                null|
|2010|1.233298120903000...|1.752972613911000...| 5.196744930080002E9|
|2011|   1.233635125179E10|1.751684120711999E10| 5.180489955329999E9|
|2012|1.245034204327000...|   1.762118501646E10| 5.170842973189999E9|
|2013|1.254475780503000...|1.780262841911000...| 5.257870614080002E9|
|2014|   1.264726299211E10|1.786939230117000...|5.2221293090599985E9|
|2015|1.256502197243999...|1.779198426312999...| 5.226962290690001E9|
|2016|   1.229711736968E10|1.746406770788999E10| 5.166950338210002E9|
|2017| 7.018850988999998E9|1.000281919499999...|       2.983968206E9|
+----+--------------------+--------------------+--------------------+

